# Effective Python

This notebook contains the best practices taken from the homonimous book.

Only some chapters are considered here, purely based on the author's ignorance.

In [1]:
import sys
print(sys.version)

3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


## It's good to unpack multiple varibles 
...rather than indexing.

Use **f-strings** to display the output.


In [2]:
def out(i):
    assert i > 0
    return list(range(i))

first, second, *others = out(10)
print(f"{first:0.3f}, {second} and {others}")

0.000, 1 and [2, 3, 4, 5, 6, 7, 8, 9]


## The following evaluate to `False`

Recall these niceties about empty strings, empty lists and `0`:

In [3]:
if []:
    print("Not printing")
elif "":
    print("Not printing")
elif 0:
    print("Not printing")
else:
    print("Got it?")

Got it?


## RAM manipulations

Direct manipulation of the memory cells.

In [4]:
a = b'123456'  # byte expression
aa = "sdojfbh"  # string expression
mv = memoryview(a)
try:
    memoryview(aa)
except TypeError:
    print("Only bytes can be manipulated in memory, not strings")
mv

Only bytes can be manipulated in memory, not strings


In [5]:
mv[2:5].tobytes()  # selecting a subsection of the RAM


b'345'

## Use ternary expression to simplify your code

In [6]:
def true_or_false(input):
    """This function will print what the
    input will evaluate to."""
    b = "True" if input else "False"
    print(b)
    
true_or_false([])

False


## Striding 
...and revertng lists.

In [7]:
## Striding
start = 1
end = 10
step = 3

test = list(range(end + 7))

print(f"Full list: {test}, \nStrided list: {test[start:end:step]}")

# A nice trick to reverse the list:
test[::-1]

Full list: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 
Strided list: [1, 4, 7]


[16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

## Sorting objects with lambda functions

Check out the abstract class and how the lambda is used for sorting (and also notice the use of `__repr__`).

In [8]:
class Obj():
    """An object

    Args:
        name (str):
            the name of the Obj
        height (float):
            the heght of the Obj
    """
    def __init__(self, name, height):
        self.name = name
        self.height = height
        
    def __repr__(self):
        """This method is used by the print function.
        It needs to return a string"""
        return f"({self.name}, {self.height})"

    def method(self):
        """whatever, not needed"""
        pass


ob1, ob2, ob3 = Obj("a", 1), Obj("bbb", 3), Obj("c", 2)

list_of_obj = [ob1, ob2, ob3]
try:
    list_of_obj.sort()
except TypeError:
    print("The sorting is not a priori defined!")

list_of_obj.sort(key=lambda x:x.name)  # we are telling to sort by the name
print(f"Sort by name: {list_of_obj}")

list_of_obj.sort(key=lambda x:x.height)  # we are telling to sort by the height
print(f"Sort by name: {list_of_obj}")

The sorting is not a priori defined!
Sort by name: [(a, 1), (bbb, 3), (c, 2)]
Sort by name: [(a, 1), (c, 2), (bbb, 3)]


## Add `__missing__` to handle missing keys in `dict`

In [9]:
class NewDict(dict):
    def __missing__(self, key):
        """This special method is called when the key
        is not defined yet."""
        self[key] = 0.0

d = NewDict()
d["key1"] = 5
d["key2"]  # this is not defined, hence filled in with `0.0`
d

{'key1': 5, 'key2': 0.0}

## Closure

Variables inside the scope of a function are not modified for outer scopes.

In [10]:

def outer_scope():
    var = 1
    def test_fun(input):
        var = 2
        return input + var
    print(f"original var: {var}\noutput of function: {test_fun(1)}")
outer_scope()

original var: 1
output of function: 3


## Impose keywords arguments

For those arguments that are really required to by keyword based, you can enforce it.

In [11]:
def fun(a, b=None, c=True, d=False):
    return a, b, c, d

def fun2(a, b=None, *, c=True, d=False):
    return a, b, c, d

fun(1,2,3,4)
try:
    fun2(1,2,3,4)
    print("Not here!")
except TypeError:
    fun2(1,2,c=3,d=4)
    print("Here!")

Here!


## Decorator for debugging

Use `functools` to wrap make sure that you can have

In [12]:
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

@trace
def function(a):
    if a < 0:
        return 0
    return function(a-1) + 3


function(10)

function((-1,), {}) -> 0
function((0,), {}) -> 3
function((1,), {}) -> 6
function((2,), {}) -> 9
function((3,), {}) -> 12
function((4,), {}) -> 15
function((5,), {}) -> 18
function((6,), {}) -> 21
function((7,), {}) -> 24
function((8,), {}) -> 27
function((9,), {}) -> 30
function((10,), {}) -> 33


33

In [13]:
help(function)  # notice that now the name is function and not wrapper

Help on function function in module __main__:

function(a)



## Iterators

Use them instead of storing the results in a list to spare memory!

Use the `__iter__` method to define the way to iterate over your data.

Use **generator expressions** to be lighter on memory!

Compose generators with `yield from`.

Use `itertools` functions, as they are very effective on iterators.

In [14]:
from collections.abc import Iterator

class IterableFoo:
    """Minimal iterable class.
    
    Args:
        lista (iterator):
            the list of inputs
    """
    def __init__(self, lista):
        self.lista = lista

    def __iter__(self):  # this is what is called when doing for x in X:
        for item in self.lista:
            yield item*4


my_list = IterableFoo([0, 1, 2, 3])
for i in my_list:
    print(i)

# define a generator
def generator(a):
    try:
        for item in a:
            yield a+3
    except TypeError:
        yield 0

# build the generator instance
it = generator([1, 2, 3])

# check whether it is an iterator
assert isinstance(it, Iterator)

not_it = generator(1)
next(not_it)
try: 
    next(not_it)
except StopIteration:
    print("We exhausted the iterator on the first call")
    
# generator expression: a.k.a. comprehension for generators
gen = (i**2 for i in range(1000000000))
print(gen)

# to compose two generators:
def my_generator():
    yield 1
    yield 2
    yield 3

# this gets data from the child() generator
def fast():
    yield from my_generator()
    
fast()

# P.S. do NOT use send, as it is complicated
def child2():
    for i in range(1_000_000):
        ii = yield i
        print("received: ", ii)
        
it = iter(child2())
print(f"First iteration: {next(it)}, sending"
      f" value: {it.send(10)}, further "
      f"iteration: {next(it)}, further iteration: {next(it)}")

# use itertools functions, as they are very effective
from itertools import combinations_with_replacement
it = combinations_with_replacement([1, 2, 3, 4], 2)
print(list(it))

0
4
8
12
We exhausted the iterator on the first call
<generator object <genexpr> at 0x7f9422c0cd58>
received:  10
received:  None
received:  None
First iteration: 0, sending value: 1, further iteration: 2, further iteration: 3
[(1, 1), (1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4), (3, 3), (3, 4), (4, 4)]


## Use hooks to treat corner cases

Hooks are functions passed to the object that can deal with corner cases. Pass hooks to deal with them.

In [15]:
# defaultdict allow for hooks to treat missing keys
from collections import defaultdict

# define the hook to handle missing keys
def missing_key():
    """returns 0 as value for a missing key"""
    return 100

d = defaultdict(missing_key)  # empty dictionary, with the policy for missing keys

d["key1"]
print(d)

defaultdict(<function missing_key at 0x7f9422c4bd08>, {'key1': 100})


# Use of `@classmethod`, `super()` and mix-in classes

In [16]:
class Mixin:
    """The Mixin class does not need any
    __init__ constructor, as it will not
    have any instance per se but will be 
    used by other class to inherit the methods.
    """
    def method1(self):
        yield 1
        yield 2

    
class Parent(Mixin):  # inherit from Mixin class
    def __init__(self, value):
        self.value = value
        self.__secret_value = value + 1  # private, cannot access

    @classmethod  # this method is bound to the class and not to self
    def method2(cls, value):  # call it withput cls, as that is automatic!
        return cls(value)  # initialise ann instance
        
class Child(Parent):
    def __init__(self, value):
        super().__init__(value)  # initialise the parent class

# initialise the child class
child = Child(10)

# results
print(f"The Child class inherits from the Parent\n class the attribute `value` "
      f"and it is {child.value}.\n The methods from the mixin class are also\n"
      f"available : {next(child.method1())},\n and {child.method2(1)}")

# P.S. Try to AVOID provate attributes
try:
    child.__secret_value
except AttributeError:
    print("Cannot access private attributes")

The Child class inherits from the Parent
 class the attribute `value` and it is 10.
 The methods from the mixin class are also
available : 1,
 and <__main__.Child object at 0x7f9422c63f28>
Cannot access private attributes


## A custom class that looks like a list

Use the containers of `collections.abc` to help you write all the functions.

In [17]:
from collections.abc import Sequence  # contains all the standard methods of lists, e.g. __iter__

class MyList(Sequence):
    def __init__(self, *args):
        self.args = args
    
    def __getitem__(self, i):
        return self.args[i]
    
    def __len__(self):
        return len(self.args)
    
lst = MyList(1,2,3,4)
print(f"Item in position 1: {lst[1]}, Length: {len(lst)}")

for i in lst:
    print(i)

Item in position 1: 2, Length: 4
1
2
3
4


## The use of `@property` or descriptors

The idea is to directly access the attributes, but to add a logic when modifying them. Also, **the syntax is exactly the same one of an attribute**!!

It is more general to use the `__set__` and `__get__` descriptors.

In [18]:
class Test:
    def __init__(self, value):
        self.value = value
    
    @property # this is the getter method
    def value(self):
        print("in the getter...")
        return self._value  # notice the underscore!
    
    @value.setter  # this is the setter method
    def value(self, new_value):
        print("in the setter...")
        if new_value > 0:
            self._value = new_value  # notice the underscore!
        else:
            raise ValueError("Need positive value")
            
tst = Test(1)  # here we set the value in te __init__, hence I expect to use the setter
print(tst.value)  # we read te value
tst.value = 2  # we set the value
print(tst.value)  # we read the value
try:
    tst.value = -2  # we try to set a negative value
except ValueError:
    print("...as expected!")

in the setter...
in the getter...
1
in the setter...
in the getter...
2
in the setter...
...as expected!


In [19]:
class Test2:
    def __init__(self):
        pass
    
    def __get__(self, instance, instance_type):
        print("in the getter...")
        return self.value

    def __set__(self, instance, new_value):
        print("in the setter...")
        if new_value > 0:
            self.value = new_value
        else:
            raise ValueError("Need positive value")

class Outer():
    grade = Test2()
        
tst2 = Outer()  # here we set the value in te __init__, hence I expect to use the setter
tst2.grade = 2  # we set the value
print(tst2.grade)  # we read the value
try:
    tst2.grade = -2  # we try to set a negative value
except ValueError:
    print("...as expected!")

in the setter...
in the getter...
2
in the setter...
...as expected!


## Dynamic creation of class attributes with `__getattribute__`

Watch out for recursions! Evey time that you access an attribute inside the `__getattribute__` you will call `__getattribute__`! Hence, do not forget to user `super()` to actually get the attribute of the instance.

In [20]:
class LazyRecord:
    def __init__(self):
        self.exists = 5
    
    def __getattribute__(self, name):  # called every time an attribute is accessed
        try:
            value = super().__getattribute__(name)  # check if the attribute exists and avoid infinite recurson
            print(f'* Found {name!r}, returning {value!r}')
            return value
        except AttributeError:
            value = f'Value for {name}'
            setattr(self, name, value)  # setting the attribute name and value
            return value
        
    def __setattr__(self, name, value):  # called every time an attribute is set
        print("setting attributes...")
        super().__setattr__(name, value)
    
ll = LazyRecord()

print(ll.__dict__)
ll.foo  # creating a new attribute!
print(ll.__dict__)
ll.foo  # creating a new attribute!
print(ll.__dict__)


setting attributes...
* Found '__dict__', returning {'exists': 5}
{'exists': 5}
setting attributes...
* Found '__dict__', returning {'exists': 5, 'foo': 'Value for foo'}
{'exists': 5, 'foo': 'Value for foo'}
* Found 'foo', returning 'Value for foo'
* Found '__dict__', returning {'exists': 5, 'foo': 'Value for foo'}
{'exists': 5, 'foo': 'Value for foo'}


## Validate all subclasses using `__init_subclass__`

Basically, in the parent class, you prepare a method that is fired every time a new class is declared (not instantiated, created!).

You can also call the method `__set_name__` to initialise Parent's class attributes when a class is created.

In [21]:
class Parent:
    def __init_subclass__(cls):
        super().__init_subclass__()
        # check presence of attribute
        print("checking...")
        print(cls)
        if not hasattr(cls, "attr"):
            raise AttributeError(f"Attribute attr not existing")
            
    def __set_name__(self, owner, name):
        """Called on class creation for each descriptor 
        when its containing class is defined, not at
        instance creation!"""
        print("Inside __set_name__...")
        self.name = name  # name of the class instance
        self.internal_name = '_' + name


class ChildOK(Parent):
    attr = 0  # required to pass validation
    def __init__(self, attr):
        self.attr = attr


class Child(Parent):
    attr = 1
    p = Parent()  # here __set_name__ is called


c = Child()
print(f"internal_name: {c.p.internal_name}")

try:
    class ChildBad(Parent):
        ttr2 = 1

except AttributeError:
    print("The validation worked")

checking...
<class '__main__.ChildOK'>
Inside __set_name__...
checking...
<class '__main__.Child'>
internal_name: _p
checking...
<class '__main__.ChildBad'>
The validation worked
